<a href="https://colab.research.google.com/github/sushirito/Hg2/blob/main/tleap_simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install AmberTools
!pip install -q condacolab
import condacolab
condacolab.install()  # Installs Conda
!mamba install -c conda-forge ambertools -y

✨🍰✨ Everything looks OK!

Looking for: ['ambertools']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [2]:
!pip install openmm
!pip install MDAnalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 88.8 MB/s eta 0:00:00


Final

In [39]:
tleap_script = """

source leaprc.protein.ff14SB

source leaprc.water.tip3p

loadAmberParams frcmod.ions234lm_126_tip3p
list
solv = createUnit solv

solvateBox solv TIP3PBOX 20.0

addIons2 solv HG 1

addIons2 solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd

quit

"""

with open('tleap.in', 'w') as f:
    f.write(tleap_script)

!tleap -f tleap.in

from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *
import sys

prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometers,
    constraints=HBonds
)

integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)

# Energy minimization
simulation.minimizeEnergy()

with open('solv_init.pdb', 'w') as f:
    PDBFile.writeFile(prmtop.topology, simulation.context.getState(getPositions=True).getPositions(), f)


simulation.reporters.append(
    PDBReporter('trajectory.pdb', 100)
)

simulation.reporters.append(
    StateDataReporter(sys.stdout, 100,
                     step=True, potentialEnergy=True, temperature=True)
)

simulation.step(5000)

with open('solv_final.pdb', 'w') as f:
    PDBFile.writeFile(prmtop.topology, simulation.context.getState(getPositions=True).getPositions(), f)


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm